In [53]:
import os

import psycopg
import pandas as pd
from numpy import linspace, random
import mlflow
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    log_loss,
)

In [54]:
random.seed(0)

##### 1. Определим глобальные перменные

In [33]:
TABLE_NAME = "users_churn"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_nikolaistepanov"
RUN_NAME = "model_grid_search"
REGISTRY_MODEL_NAME = "churn_model_nikolaistepanov_prepared"

##### 2. Заберем данные

In [34]:
connection = {"sslmode": "verify-full", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
    "dbname": os.getenv("POSTGRES_DBNAME"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
}

connection.update(postgres_credentials)

In [35]:
with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

##### 3. Разделим данные на train, test по `begin_date`

In [36]:
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "begin_date"
stratify_column = ["type"]
test_size = 0.2

In [37]:
df = df.sort_values(by=[split_column])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    test_size=test_size,
    shuffle=False,
)

In [39]:
print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

Размер выборки для обучения: (5634, 3)
Размер выборки для теста: (1409, 3)


##### 4. Определим гиперпараметры модели, которые хотим подобрать, и диапазоны возможных значений для них

In [40]:
loss_function = "Logloss"
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

params = {
    "learning_rate": linspace(start=0.001, stop=0.1, num=5),
    "depth": [2, 5],
    "l2_leaf_reg": linspace(start=0.1, stop=5, num=5),
    "random_strength": linspace(start=0.1, stop=5, num=5),
}

In [41]:
model = CatBoostClassifier(
    iterations=iterations, 
    loss_function=loss_function, 
    random_seed=random_seed, 
    task_type=task_type,
    verbose=verbose,
)

##### 5. Определим объект класса GridSearch

In [42]:
cv = GridSearchCV(
    estimator=model, 
    param_grid=params, 
    n_jobs=-1, 
    cv=2
)

##### 6. Запустим побор гиперпараметров

In [43]:
%%time

clf = cv.fit(X_train, y_train)

CPU times: user 953 ms, sys: 519 ms, total: 1.47 s
Wall time: 23.3 s


##### 7. Соберем аналитику по подбору и залогируем в `MLFlow`

In [44]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "..."
os.environ["AWS_ACCESS_KEY_ID"] = "..."
os.environ["AWS_SECRET_ACCESS_KEY"] = "..."

In [45]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [46]:
cv_results = pd.DataFrame(clf.cv_results_)

best_params = clf.best_params_

In [47]:
cv_results.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_l2_leaf_reg,param_learning_rate,param_random_strength,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.296663,0.004098,0.005787,0.000398,2,0.1,0.001,0.1,"{'depth': 2, 'l2_leaf_reg': 0.1, 'learning_rat...",0.785942,0.683351,0.734647,0.051296,33
1,0.287730,0.027295,0.004534,0.001132,2,0.1,0.001,1.325,"{'depth': 2, 'l2_leaf_reg': 0.1, 'learning_rat...",0.785942,0.733049,0.759496,0.026447,17


In [48]:
model = CatBoostClassifier(
    iterations=iterations, 
    loss_function=loss_function, 
    random_seed=random_seed, 
    task_type=task_type,
    verbose=verbose,
    **best_params,
)

In [49]:
model.fit(X_train, y_train)

In [50]:
prediction = model.predict(X_test)
probas = model.predict_proba(X_test)[:, 1]

In [57]:
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

metrics["mean_fit_time"] = cv_results["mean_fit_time"].mean()
metrics["std_fit_time"] = cv_results["std_fit_time"].mean()
metrics["mean_test_score"] = cv_results["mean_test_score"].mean()
metrics["std_test_score"] = cv_results["std_test_score"].mean()

metrics["best_score"] = clf.best_score_

best_params = clf.best_params_

/Users/nikolaistepanov/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
metrics

{'err1': 0.0,
 'err2': 0.0,
 'auc': 0.7064808187659471,
 'precision': 0.0,
 'recall': 0.0,
 'f1': 0.0,
 'logloss': 16.96021447621339,
 'mean_fit_time': 0.3372681589126587,
 'std_fit_time': 0.03691943073272705,
 'mean_test_score': 0.6290067447639333,
 'std_test_score': 0.15715441959531415,
 'best_score': 0.796059637912673}

In [55]:
best_params

{'depth': 2,
 'l2_leaf_reg': 0.1,
 'learning_rate': 0.001,
 'random_strength': 5.0}

In [52]:
pip_requirements="../requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metrics(metrics)
    mlflow.log_params(best_params)
    cv_info = mlflow.sklearn.log_model(cv, artifact_path="cv")
    model_info = mlflow.catboost.log_model(
        cb_model=model, 
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name=REGISTRY_MODEL_NAME,
        await_registration_for=60,
        pip_requirements=pip_requirements,
    )

/Users/nikolaistepanov/.pyenv/versions/3.11.3/lib/python3.11/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
/Users/nikolaistepanov/.pyenv/versions/3.11.3/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before 